## Convert filter_blast function to use pandas

In [1]:
import pandas as pd
import numpy as np

In [3]:
import sys
sys.path.append('/Users/hcross/analysis/git_projects/genbanking/gb_parsers/')
from blast_parsers import blast_parser

In [4]:
# now try to re-parse the small file
blast_tab = blast_parser('example_blast_output_default_format.txt')

In [5]:
blast_tab.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
1,MA_19953_MA_19953.g3.t1,gi|60392921|sp|P11369.2|LORF2_MOUSE,37.681,69,42,1,209,276,510,578,3.920000e-05,50.1
2,MA_19953_MA_19953.g4.t1,gi|141475|sp|P14381.1|YTX2_XENLA,22.966,762,480,22,328,1012,128,859,2.240000e-35,150.0
3,MA_19953_MA_19953.g5.t1,gi|81615471|sp|Q6LU24.1|PUR4_PHOPR,31.868,91,57,2,13,98,971,1061,2.400000e-02,40.0
4,MA_19953_MA_19953.g6.t1,gi|81960279|sp|Q914H8.1|Y054_SIFVH,19.880,166,122,4,17,175,176,337,9.800000e-02,37.7


### Build a function that includes the variables from original, but adding pandas function

#### Use commands like these:

```python
blast2 = blast_table.loc[blast_table['pident'] > 35]

blast3 = blast_table[(blast_table['pident'] > 35) & (blast_table['length'] > 100)]
```


In [10]:
great = '>'
blast2 = blast_tab.loc[blast_tab['pident'] %s 35] % (great)

SyntaxError: invalid syntax (<ipython-input-10-c8530eaa4353>, line 2)

In [11]:
string = "blast2 = blast_tab.loc[blast_tab['pident'] > 35]"
exec(string)

In [12]:
blast2.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
1,MA_19953_MA_19953.g3.t1,gi|60392921|sp|P11369.2|LORF2_MOUSE,37.681,69,42,1,209,276,510,578,3.920000e-05,50.1
5,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,64.103,39,14,0,114,152,530,568,2.210000e-07,55.1
6,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,50.000,40,20,0,52,91,295,334,2.000000e-03,43.5
9,MA_19953_MA_19953.g9.t1,gi|135915|sp|P28493.1|PR5_ARATH,65.948,232,75,4,1,232,12,239,5.910000e-99,294.0


**In order to be flexible with testing fields, have to create strings, and then convert it to command using exec(command_string)** 
found here:


http://stackoverflow.com/questions/701802/how-do-i-execute-a-string-containing-python-code-in-python

In [84]:
def filter_blast(blastDF, blast_parameters):
    """Filter a blast result dictionary based on variable parameters using pandas"""
    ## >> have to add more to following
    field_match = {'SUB_LENGTH':'slen','MIN_QSTART':'qstart','MAX_QSTART':'qstart','MIN_QEND':'qend','MAX_QEND':'qend','MIN_SEND':'send','MAX_SEND':'send','MIN_BITSCORE':'bitscore','MIN_POSITIVE':'positive','MAX_GAPOPEN':'gapopen','MAX_GAPS':'gaps','IN_TITLE':'stitle','IN_SSEQ':'sseq','MIN_SSTART':'sstart','MAX_SSTART':'sstart','MIN_QCOV':'qcovs','MIN_LENGTH':'length','MIN_PCT_ID':'pident','MAX_MISMATCH':'mismatch','MAX_EVALUE':'evalue','IN_SUBJECT_ID':'sseqid'}
    mins = ['MIN_LENGTH','MIN_PCT_ID','MIN_SSTART','MIN_QCOV','SUB_LENGTH','MIN_QSTART','MIN_QEND','MIN_SEND','MIN_BITSCORE','MIN_POSITIVE'] 
    maxes = ['MAX_MISMATCH','MAX_EVALUE','MAX_SSTART','MAX_QSTART','MAX_QEND','MAX_SEND','MAX_GAPOPEN','MAX_GAPS'] 
    containers = ['IN_SUBJECT_ID','IN_TITLE','IN_SSEQ']
    filtered_blast = {}
    blast_parameters.pop('FORMAT')
    num_conditions = len(blast_parameters)
    # for now have to build it as a string and then run 
    string_list = []
    for k,v in blast_parameters.iteritems():
        # blast2 = blast_table.loc[blast_table['pident'] > 35]
        blast_field = field_match[k]
        string = "blastDF[blast_field]"
        if k in mins:
            #string = string + "> "+v+"]"
            string = string + "> %s" %(v)
            #if hit_dict[blast_field] > v:   
        elif k in maxes:
            string = string + "< %s" %(v)
            #if hit_dict[blast_field] < v:
                
        elif k in containers:
            # bb5 = bb[bb['stitle'].str.contains('Maireana')]
            #string = string+".str.contains("+v+")]"
            string = string+".str.contains('%s')" % (v)
            #if v in hit_dict[blast_field]: # need to fix this, was in v
        string_list.append(string)
    if num_conditions == 1:
        cmd_string = string_list[0]
        final_string = "filt_blastDF = blastDF["+cmd_string+"]"
        exec(final_string)
        
    # for multiple conditions:
    #blast3 = blast_table[(blast_table['pident'] > 35) & (blast_table['length'] > 100)]
    else:
        initial = "filt_blastDF = blastDF["
        for cmd in string_list:
            new_string = "("+cmd+")"
            initial = initial+new_string+"&"
        semifinal = initial[:-1]
        final = semifinal+"]"
        exec(final)
    return filt_blastDF
    

In [85]:
# try a simple one 
para_dict = {'MIN_LENGTH':100, 'FORMAT':'default'}

In [86]:
filt1 = filter_blast(blast_tab, para_dict)

In [87]:
filt1.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
2,MA_19953_MA_19953.g4.t1,gi|141475|sp|P14381.1|YTX2_XENLA,22.966,762,480,22,328,1012,128,859,2.240000e-35,150.0
4,MA_19953_MA_19953.g6.t1,gi|81960279|sp|Q914H8.1|Y054_SIFVH,19.880,166,122,4,17,175,176,337,9.800000e-02,37.7
9,MA_19953_MA_19953.g9.t1,gi|135915|sp|P28493.1|PR5_ARATH,65.948,232,75,4,1,232,12,239,5.910000e-99,294.0
13,MA_10432594_MA_10432594.g13.t1,gi|134034966|sp|Q99315.3|YG31B_YEAST,28.252,1207,672,27,418,1453,267,1450,8.210000e-132,455.0


In [64]:
para_dict2 = {'MAX_MISMATCH':30, 'FORMAT':'default'}

In [65]:
filt2 = filter_blast(blast_tab, para_dict2)

filt_blastDF = blastDF[blastDF[blast_field]< 30]


In [66]:
filt2.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
5,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,64.103,39,14,0,114,152,530,568,2.210000e-07,55.1
6,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,50.000,40,20,0,52,91,295,334,2.000000e-03,43.5
11,MA_10434223_MA_10434223.g11.t1,gi|296439825|sp|A7MC48.2|SHSA9_XENTR,42.857,28,16,0,116,143,1,28,5.000000e+00,30.4
12,MA_10434223_MA_10434223.g12.t1,gi|11135132|sp|O65049.1|TRXH_PICMA,98.980,98,1,0,18,115,28,125,4.660000e-63,194.0
19,MA_10432594_MA_10432594.g18.t1,gi|75215641|sp|Q9XI47.1|SHH1_ARATH,38.462,26,15,1,3,27,65,90,5.200000e+00,28.5


In [88]:
para_dict3 = {'IN_SUBJECT_ID': 'ARATH', 'FORMAT':'default'}

In [89]:
filt3 = filter_blast(blast_tab, para_dict3)

In [90]:
filt3.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
9,MA_19953_MA_19953.g9.t1,gi|135915|sp|P28493.1|PR5_ARATH,65.948,232,75,4,1,232,12,239,5.910000e-99,294.0
17,MA_10432594_MA_10432594.g17.t1,gi|75154272|sp|Q8L817.1|ATCA7_ARATH,51.685,178,85,1,44,220,36,213,2.070000e-62,206.0
18,MA_10432594_MA_10432594.g17.t1,gi|75154272|sp|Q8L817.1|ATCA7_ARATH,45.038,131,54,1,219,331,143,273,2.070000e-30,121.0
19,MA_10432594_MA_10432594.g18.t1,gi|75215641|sp|Q9XI47.1|SHH1_ARATH,38.462,26,15,1,3,27,65,90,5.200000e+00,28.5


In [35]:
string2 = "blast3 = blast_tab[blast_tab['sseqid'].str.contains('ARATH')]"
exec(string2)

In [36]:
blast3.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
9,MA_19953_MA_19953.g9.t1,gi|135915|sp|P28493.1|PR5_ARATH,65.948,232,75,4,1,232,12,239,5.910000e-99,294.0
17,MA_10432594_MA_10432594.g17.t1,gi|75154272|sp|Q8L817.1|ATCA7_ARATH,51.685,178,85,1,44,220,36,213,2.070000e-62,206.0
18,MA_10432594_MA_10432594.g17.t1,gi|75154272|sp|Q8L817.1|ATCA7_ARATH,45.038,131,54,1,219,331,143,273,2.070000e-30,121.0
19,MA_10432594_MA_10432594.g18.t1,gi|75215641|sp|Q9XI47.1|SHH1_ARATH,38.462,26,15,1,3,27,65,90,5.200000e+00,28.5


In [91]:
# now try with multis
para_dict4 = {'MIN_LENGTH':10, 'MAX_MISMATCH':50, 'FORMAT':'default'}

In [92]:
filt4 = filter_blast(blast_tab, para_dict4)

In [93]:
filt4.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
1,MA_19953_MA_19953.g3.t1,gi|60392921|sp|P11369.2|LORF2_MOUSE,37.681,69,42,1,209,276,510,578,3.920000e-05,50.1
5,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,64.103,39,14,0,114,152,530,568,2.210000e-07,55.1
6,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,50.000,40,20,0,52,91,295,334,2.000000e-03,43.5
7,MA_19953_MA_19953.g7.t1,gi|130582|sp|P10978.1|POLX_TOBAC,30.189,53,37,0,1,53,112,164,4.400000e+00,32.3
10,MA_10434223_MA_10434223.g10.t1,gi|146291076|sp|Q7LHG5.2|YI31B_YEAST,31.667,60,39,1,231,290,663,720,4.830000e-04,45.1


In [80]:
blast_tab.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,MA_19953_MA_19953.g2.t1,gi|71153422|sp|Q9LLL8.1|SBT4E_ARATH,35.537,121,65,5,280,394,348,461,8.070000e-07,54.7
1,MA_19953_MA_19953.g3.t1,gi|60392921|sp|P11369.2|LORF2_MOUSE,37.681,69,42,1,209,276,510,578,3.920000e-05,50.1
2,MA_19953_MA_19953.g4.t1,gi|141475|sp|P14381.1|YTX2_XENLA,22.966,762,480,22,328,1012,128,859,2.240000e-35,150.0
3,MA_19953_MA_19953.g5.t1,gi|81615471|sp|Q6LU24.1|PUR4_PHOPR,31.868,91,57,2,13,98,971,1061,2.400000e-02,40.0
4,MA_19953_MA_19953.g6.t1,gi|81960279|sp|Q914H8.1|Y054_SIFVH,19.880,166,122,4,17,175,176,337,9.800000e-02,37.7
